# Testing federated learning
The purpose of this notebook is to run a data analysis pipeline on vantage 6.

The outcome of the pipeline is irrelevant so far.  This notebook is mainly a POC for creating a data analysis client-side and using it to train a model on a federated dataset.

In [1]:
import vantage6.client as vtgclient
from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression, GammaRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import pipeline
from sklearn.impute import SimpleImputer
from run_task import print_result, print_logs
import numpy as np

In [3]:
HOST = 'http://vserver'
PORT = 5000
IMAGE ='localhost:5000/v6-carrier-py'

USERNAME = 'admin'
PASSWORD = 'admin'

COLLABORATION_ID = 1
MASTER_ORGANIZATION = [1]
RPC_ORGANIZATIONS = [2,3, 6]

KEYS = ['GBAGeboorteJaar', 'GBAGeboorteMaand', 'GBAGeboorteDag', 'GBAGeslacht',
        'GBAPostcode', 'GBAHuisnummer', 'GBAToev']


FEATURES = ['height', 'weight', 'bmi', 'Age', 'n_smokingcat4', 'WOZ', 'N_ALCOHOL_CAT',
            'ZVWKOPHOOGFACTOR_2010', 'ZVWKHUISARTS_2010', 'ZVWKFARMACIE_2010', 'ZVWKZIEKENHUIS_2010', 'ZVWKPARAMEDISCH_2010', 'ZVWKZIEKENVERVOER_2010',
        'ZVWKBUITENLAND_2010', 'ZVWKOVERIG_2010', 'ZVWKEERSTELIJNSPSYCHO_2010', 'ZVWKGGZ_2010',
        'ZVWKHULPMIDDEL_2010', 'ZVWKOPHOOGFACTOR_2011', 'ZVWKHUISARTS_2011']
TARGET = 'N_CVD'
client = vtgclient.Client(HOST, PORT)
client.authenticate(USERNAME, PASSWORD)
client.setup_encryption(None)

In [4]:
# Retrieve columns names to check if a simple algorithm succeeds
task = client.post_task('column_names', image=IMAGE, collaboration_id=COLLABORATION_ID,
                        organization_ids=RPC_ORGANIZATIONS,
                        input_={'method': 'column_names', 'master': False})

In [6]:
# It will take a few seconds before results are returned
result = client.get_results(task_id=task['id'])
print_result(result)
print_logs(result)

0: ['Unnamed: 0', 'GBAGeboorteJaar', 'GBAGeboorteMaand', 'GBAGeboorteDag', 'GBAGeslacht', 'GBAPostcode', 'GBAHuisnummer', 'GBAToev', 'VISIT1_DATE', 'N_DM_DUR_2', 'N_GTS_WHO', 'N_Diabetes_WHO_2', 'N_DIABETES_2', 'N_T2DM_new_diagn', 'SEX', 'Age', 'N_Education_3cat', 'Employment_status', 'Occupational_category', 'Income_equivalent', 'n_smokingcat4', 'WOZ', 'N_ALCOHOL_CAT', 'NIT_kcal', 'medGRscore_TR', 'height', 'weight', 'bmi', 'N_CVD', 'med_depression', 'PHQ9_depr', 'MINIcurrdepr', 'MINIlifedepr', 'N_HT', 'TOTALPA', 'MVPA', 'MEAN_VALID_MIN_SLEEP_T', 'MEAN_SED_MIN_WAKE_T', 'MEAN_STAND_MIN_WAKE_T', 'MEAN_STEP_MIN_WAKE_T', 'waist', 'hip', 'SF36_Q7', 'SF36_Q10', 'Mobility_lim']
1: ['Unnamed: 0', 'GBAGeboorteJaar', 'GBAGeboorteMaand', 'GBAGeboorteDag', 'GBAGeslacht', 'GBAPostcode', 'GBAHuisnummer', 'GBAToev', 'VISIT1_DATE', 'N_DM_DUR_2', 'N_GTS_WHO', 'N_Diabetes_WHO_2', 'N_DIABETES_2', 'N_T2DM_new_diagn', 'SEX', 'Age', 'N_Education_3cat', 'Employment_status', 'Occupational_category', 'Income_

In [13]:
pipe = pipeline.make_pipeline(SimpleImputer(missing_values=0), StandardScaler(), LinearRegression())

In [14]:
task = client.post_task('fit_pipeline', image=IMAGE, collaboration_id=COLLABORATION_ID,
                        organization_ids=MASTER_ORGANIZATION,
                        input_={'method': 'fit_pipeline', 'master': True,
                            'kwargs': {'pipe': pipe, 'features': FEATURES, 'target': TARGET,
                                      'keys': KEYS, 'exclude_orgs': MASTER_ORGANIZATION + [3]}})

In [16]:
result = client.get_results(task_id=task['id'])
print_result(result)
print_logs(result)

0: None
Log for organization 1
None


In [52]:
# The actual result of the algorithm
result[0]['result']